# Параллельное программирование

## 1. О параллелизме

**Параллельные вычисления** — способ решать задачи с помощью нескольких вычислительных модулей. Распараллеливать можно независимые части в задаче или её решении.

Выделяют два вида параллелизма.
- *Параллелизм задач* — когда есть независимы подзадачи. Разные инструкции выполняются на одних и тех же данных.
- *Параллелизм данных*, когда есть независимые участки данных. Одинаковые инструкции выполняются на разных кусочках данных.

#### Имеет ли смысл распараллеливать задачу? Закон Амдала (англ. Amdahl’s Law)
Закон Амдала показывает во сколько раз меньше времени потребуется параллельной программе для решения задачи по сравнению с последовательной. 
$$
\text{Speed-up}=\frac{1}{S+P/n}
$$

### Модели параллельного программирования


Есть две программные модели параллельного программирования:
- модель с общей памятью,
- модель с распределённой памятью.

#### Модель с общей памятью (англ. Shared Memory Model)

Различают два вида:
- С потоками (англ. threads). OpenMP, Posix Threads.
- Без потоков.

#### Модель с распределенной памятью (англ. Distributed Memory Model)

Модель передачи сообщений (англ. Message Passing Model MPI).

- Асинхронная передача сообщений. Actor model
- Синхронная передача сообщений. Communicating sequential processes

### Способы синхронизации

- Семафоры
- Мьютекс (англ. mutex = mutual exclusion) aka Lock.
- Критические секции

### Почитать
- [Такие удивительные семафоры](https://habr.com/en/post/261273/)

## 2. Threads

Поток (thread) — подзадача процесса операционной системы. Потоки одного процесса делят между собой его общую память. В Python потоки являются системными объяктами, то есть ими управляет операционная система.

In [1]:
from threading import Thread

In [22]:
def subproc(n: int):
    [print(i) for i in range(n)]
    
thread1 = Thread(target=subproc, args=(5,))
thread2 = Thread(target=subproc, args=(5,))

thread1.start()
thread2.start()
thread1.join()
thread2.join()

0
1
2
3
4
0
1
2
3
4


In [40]:
class MyThread(Thread):
    def __init__(self, n:int):
        super().__init__(name=f"Up to {n}")
        self.n = n
    def run(self):
        [print(i) for i in range(self.n)]
        
thread1 = MyThread(5)
thread2 = MyThread(5)

thread1.start()
thread2.start()
thread1.join()
thread2.join()

0
1
2
3
4
0
1
2
3
4


### Демонизация потоков

Процесс будет работать до тех пор, пока все потоки не будут завершены. Если надо по явному завершению процесса прекратить работу всех дочерних потоков, то их надо сделать демонами.

In [31]:
def subproc(n: int):
    [print(i) for i in range(n)]
    
thread1 = Thread(target=subproc, args=(5,), daemon=True)
thread2 = Thread(target=subproc, args=(5,), daemon=True)

thread1.start()
thread2.start()
thread1.join()
thread2.join()

0
1
2
3
4
0
1
2
3
4


### GIL (Global Interpreter Lock)

GIL — глобальный lock интерпретатора. Это особенность реализации Python из-за которой нельзя одновременно использовать несколько процессоров для потоков. Все параллельные вычисления в Python реализуются с помощью модулей.

### Почитать
- [Как устроен GIL в Python](https://habr.com/en/post/84629/)

### Блокировка ресурсов

#### Мьютекс

In [41]:
from threading import Lock

In [47]:
mutex = Lock()

mutex.acquire()
print(mutex)
# Работа с общими ресурсами
mutex.release()
print(mutex)

<locked _thread.lock object at 0x109b52a80>
<unlocked _thread.lock object at 0x109b52a80>


In [64]:
x = 0
def fun_inc(n:int):
    global x
    for _ in range(n):
        x += 1
    
def fun_dec(n:int):
    global x
    for _ in range(n):
        x -= 1

thread1 = Thread(target=fun_inc, args=(5000,))
thread2 = Thread(target=fun_dec, args=(5000,))

thread1.start()
thread2.start()
thread1.join()
thread2.join()

print(x)

0


In [76]:
try:
    mutex.acquire()
    # Работа с общими ресурсами

except:
    # Обрабатываем исключения
    pass

finally:
    mutex.release()

In [ ]:
with

#### Рекурсивный мьютекс

In [74]:
from threading import RLock

#### Семафор

In [67]:
from threading import Semaphore

In [69]:
s = Semaphore(10)

s.acquire()
print(s)
# Работа с общими ресурсами
s.release()
print(s)

#### События

In [70]:
from threading import Event

In [73]:
e = Event()

#e.wait() # Ждём, когда кто-нибудь захватит флаг
print(e)
e.set() # Ставим флаг
print(e)
# Работа с общими ресурсами
e.clear() # Снимаем флаг и ждём нового
print(e)

#### Условия

## 3. Processes

## 4. MPI

## Бонус: аппаратный взгляд на паралелизм

### Классификация Флина

[Классификация Флина](https://en.wikipedia.org/wiki/Flynn%27s_taxonomy)

![Flynn's Taxonomy](img/parallel-architectures.png)